# Рейтинговая рекомендательная система
_______

Начнём программировать твою первую неперсонализированную рекомендательную систему. 

Наш план такой:
 - Загрузим **датасет из файла movies_metadata_fixed.csv**. 
 - Датасет находится в «сыром» виде, поэтому сделаем **предобработку данных**. Другими словами, подготовим все необходимые колонки из датасета и приведем в удобный формат.
 - Напишем функции, которые будут оценивать каждый фильм по некоторой **формуле**. Формул будет несколько,поэтому сможем сравнить разные рейтинги, оценить разницу и выбрать лучший вариант.
 
 _____

Для обработки и анализа датасета нужны библиотеки Pandas и Ast, импортируем их

In [80]:
import warnings
warnings.filterwarnings('ignore')

**Pandas** позволит загружать и трансформировать данные при помощи простых и понятных функций

In [81]:
#загрузи pandas под псевдонимом pd
import pandas as pd

Библиотека **Ast** и пакет **Literal_eval** пригодятся для извлечения информации из «сырых» форматов данных

In [82]:
from ast import literal_eval

Загружаем датасет из файла **movies_metadata_fixed.csv**

In [83]:
path_to_dataset = 'movies_metadata_fixed.csv'#укажи верный путь к файлу на своем устройстве
dataset = pd.read_csv(path_to_dataset)
dataset.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92


По строкам в датасете хранятся отдельные фильмы. После загрузки файла, ты увидишь, что в столбцах для каждого фильма доступна разная информация: бюджет, название, актеры, режиссеры, обзор, язык и т.д. 

В первом задании нас интересуют несколько переменных: жанр(genres), год выпуска(year), средний рейтинг фильма(vote_average) и количество оценок фильма(vote_count).

____

### Жанры

Начнем с колонки "жанры" и посмотрим, как она выглядит:

In [84]:
#посмотри на значения элементов в колонке genres
dataset.genres.iloc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

Что видим? Строка, в которой записан список словарей. Очевидно, что нам не нужно столько информации, поэтому преобразуем это поле в обычный список. Здесь пригодится пакет literal_eval, который может обрабатывать подобные строки. Например:

In [85]:
literal_eval(dataset['genres'].iloc[0])

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

На выходе **Literal_eval** всё тот же список словарей, но теперь он не в виде строки, а в виде объектов языка Python.

Чтобы получить список жанров из такой структуры, нужно обратиться к полю **name** в каждом элементе списка словарей.

In [86]:
[i['name'] for i in literal_eval(dataset['genres'].iloc[0])]

['Animation', 'Comedy', 'Family']

Теперь преобразуем поле **genres** целиком так же, как и в примере выше 

In [87]:
dataset['genres'] = dataset['genres'].fillna('[]')\
                                     .apply(literal_eval)\
                                     .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [88]:
dataset['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

Выясним, фильмы каких жанров содержатся в датасете. 

In [89]:
import functools
available_genres = functools.reduce(lambda x,y: set(x).union(set(y)), dataset.genres.tolist())
available_genres

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

Отлично, мы получили в распоряжение полный список жанров фильмов в датасете, переходим к следующей колонке.

___


###  Год выпуска

Год выпуска фильма можно извлечь из поля release_date. Сделаем это с помощью функций Pandas.

In [92]:
dataset['year'] = pd.to_datetime(dataset.release_date).dt.year
dataset.year.dtype

dtype('float64')

___

### Оценки пользователей

Что дальше? Для построения рейтинга нам пригодятся 2 колонки с оценками пользователей: vote_average и vote_count. Убедимся, что эти колонки в числовом формате, а если нет, преобразуем их.

In [93]:
dataset[['vote_count', 'vote_average']].dtypes

vote_count        int64
vote_average    float64
dtype: object

Их типы int64 и float64. Все в порядке.

____

### Формула ранжирования

Перед тем как перейти к созданию формул, оставим только необходимые для работы поля в датасете, а другие удалим за ненадобностью.

In [94]:
def process_dataset(df): 
    # Удаление фильмов, для которых отсутствует информация о среднем рейтинге и кол-ве проголосовавших пользователей. 
    # Используй метод notnull() для интересующих колонок
    ok_rows = ((dataset.vote_count.notnull()) & (dataset.vote_average.notnull()))
    rank_dataset = dataset[ok_rows]
    
    # Оставим в нашем датасете только колонки: 'title', 'year', 'vote_count', 'vote_average', 'genres'
    usecols_ = ['title', 'year', 'vote_count', 'vote_average', 'genres', 'adult']
    rank_dataset = dataset[usecols_]
    
    return rank_dataset

In [96]:
dataset  = process_dataset(dataset.copy())
dataset.head()

,title,year,vote_count,vote_average,genres,adult
0,Toy Story,1995.0,5415,7.7,"[Animation, Comedy, Family]",False
1,Jumanji,1995.0,2413,6.9,"[Adventure, Fantasy, Family]",False
2,Grumpier Old Men,1995.0,92,6.5,"[Romance, Comedy]",False
3,Waiting to Exhale,1995.0,34,6.1,"[Comedy, Drama, Romance]",False
4,Father of the Bride Part II,1995.0,173,5.7,[Comedy],False


Теперь гораздо удобнее работать с датасетом. Приступаем к разработке рекомендательной системы. 

Главный элемент для систем неперсональных рекомендаций — это формула для составления рейтинга фильма. Мы запрограммируем несколько вариантов такой формулы и отсортируем все фильмы по показателю рейтинга. 

У тебя может возникнуть вопрос:

— Если есть колонка vote_average, где посчитан средний рейтинг фильма, то почему бы просто не использовать это поле?

Действительно, это самый простой вариант (и даже посчитанный за нас). Давай посмотрим, что получится, если использовать такую схему для формирования рейтинга (ранжирования) фильмов.

In [97]:
def simple_formula(input_ds):
    """
    Films are ranked by vote_average
    """
    #извлекаем среднюю оценку vote_average
    v_avr = input_ds.vote_average
    return v_avr

def recommend_simple(input_films, n_items):
    rank = input_films.copy()
    #применяем простую формулу ко всем строкам при помощи лямбда-функции
    rank['formula'] = rank.apply(lambda x: simple_formula(x), axis='columns')
    rank.sort_values('formula', ascending=False, inplace=True)
    return rank.head(n_items).reset_index(drop=True)

recommend_simple(dataset, 10)

,title,year,vote_count,vote_average,genres,adult,formula
0,Zig Zag Story,1983.0,1,10.0,"[Drama, Comedy]",False,10.0
1,Tall Story,1960.0,1,10.0,[Comedy],False,10.0
2,Birch Interval,1976.0,1,10.0,[Drama],False,10.0
3,Mad at the Moon,1992.0,1,10.0,[],False,10.0
4,The White Shadow,1924.0,1,10.0,[],False,10.0
5,Forever,2006.0,1,10.0,[Documentary],False,10.0
6,Chilly Scenes of Winter,1979.0,1,10.0,"[Comedy, Drama, Romance]",False,10.0
7,The Great Kidnapping,1973.0,1,10.0,[Crime],False,10.0
8,Backyard Dogs,2001.0,1,10.0,"[Action, Comedy]",False,10.0
9,"Oh, Bomb!",1964.0,1,10.0,"[Music, Comedy, Action, Crime]",False,10.0


Какой получился результат? Мы получили список фильмов с самым высоким рейтингом, но легко заметить, что этот рейтинг был сформирован из оценок только одного пользователя. Думаю, ты бы не стал смотреть ничего из этого списка.

Как справится с этой незадачей? Простой способ: учесть количество средних оценок для каждого фильма.

In [69]:
def better_formula(input_ds):
    #извлекаем количество оценивших vote_count
    v_count = input_ds.vote_count
    #извлекаем среднюю оценку vote_average
    v_avr = input_ds.vote_average
    #возвращаем их произведение, это самый простой способ учесть количество
    return v_count * v_avr

def recommend_better(input_films, n_items):
    rank = input_films.copy()
    #применим формулу ко всем строкам при помощи лямбда-функции
    rank['formula'] = rank.apply(lambda x: better_formula(x), axis='columns')
    rank.sort_values('formula', ascending=False, inplace=True)
    return rank.head(n_items).reset_index(drop=True)

recommend_better(dataset, 10)

,title,year,vote_count,vote_average,genres,formula
0,Inception,2010.0,14075,8.1,"[Action, Thriller, Science Fiction, Mystery, A...",114007.5
1,The Dark Knight,2008.0,12269,8.3,"[Drama, Action, Crime, Thriller]",101832.7
2,Interstellar,2014.0,11187,8.1,"[Adventure, Drama, Science Fiction]",90614.7
3,The Avengers,2012.0,12000,7.4,"[Science Fiction, Action, Adventure]",88800.0
4,Avatar,2009.0,12114,7.2,"[Action, Adventure, Fantasy, Science Fiction]",87220.8
5,Deadpool,2016.0,11444,7.4,"[Action, Adventure, Comedy]",84685.6
6,Fight Club,1999.0,9678,8.3,[Drama],80327.4
7,Django Unchained,2012.0,10297,7.8,"[Drama, Western]",80316.6
8,Guardians of the Galaxy,2014.0,10014,7.9,"[Action, Science Fiction, Adventure]",79110.6
9,Pulp Fiction,1994.0,8670,8.3,"[Thriller, Crime]",71961.0


Видим, что результат стал гораздо лучше, в топе рейтинга находятся довольно известные и популярные фильмы. 
___

Можно ли сделать лучше? Точно, **ДА**. 

До этого мы готовы были порекомендовать любой фильм доступный в датасете, даже если его рейтинг сформирован оценкой единственного человека. 

Однако, наша задача - порекомендовать один или несколько фильмов, у которых по-настоящему высокий рейтинг, даже если сейчас его оценили всего несколько человек. Каким был бы рейтинг такого фильма, если бы его посмотрело больше людей? Мы можем только предполагать, поэтому нам остается лишь вариант со средним арифметическим, потому что по статистике, это будет наилучшая оценка среднего.

Если есть отзывы о фильме и какая-то начальная средняя оценка (даже состоящая из 1-го голоса), нечестно её не учитывать.

Для нашей финальной формулы мы попробуем учесть и среднее арифметическое, и текущую оценку одновременно. Но как? Ответ: взвешенное среднее. Это такое среднее, которое учитывает важность отдельных элементов в формуле. В нашем случае, чем больше людей (vote_count) оставили оценку фильму, тем важнее эта информация для общего рейтинга. 

In [72]:
def the_best_formula(x, Q, C):
    v = float(x['vote_count'])
    R = x['vote_average']
    #попробуй самостоятельно рассчитать формулу для взвешенного среднего
    return (v/(v+Q) * R) + (Q/(Q+v) * C)

def recommend_best(input_films, n_items):
    rank = input_films.copy()
    
    C = rank.vote_average.mean()
    Q = rank.vote_count.quantile(0.95)
    print("Minimum number of votes: {}. Average overall rating: {}".format(int(Q),C))
    
    #применим лучшую формулу ко всем строкам при помощи лямбда-функции
    rank['formula'] = rank.apply(lambda x: the_best_formula(x, Q, C), axis='columns')
    rank.sort_values('formula', ascending=False, inplace=True)
    return rank.head(n_items).reset_index(drop=True)

recommend_best(dataset, 10)

Minimum number of votes: 433. Average overall rating: 5.618217011635541


,title,year,vote_count,vote_average,genres,formula
0,The Shawshank Redemption,1994.0,8358,8.5,"[Drama, Crime]",8.357778
1,The Godfather,1972.0,6024,8.5,"[Drama, Crime]",8.306376
2,The Dark Knight,2008.0,12269,8.3,"[Drama, Action, Crime, Thriller]",8.208397
3,Fight Club,1999.0,9678,8.3,[Drama],8.184925
4,Pulp Fiction,1994.0,8670,8.3,"[Thriller, Crime]",8.172184
5,Forrest Gump,1994.0,8147,8.2,"[Comedy, Drama, Romance]",8.069450
6,Schindler's List,1993.0,4436,8.3,"[Drama, History, War]",8.061058
7,Whiplash,2014.0,4376,8.3,[Drama],8.058077
8,Spirited Away,2001.0,3968,8.3,"[Fantasy, Adventure, Animation, Family]",8.035654
9,The Empire Strikes Back,1980.0,5998,8.2,"[Adventure, Action, Science Fiction]",8.025831


Когда наша формула рейтинга уже не так плоха, пофантазируем, какие еще параметры фильма могут интересовать человека?

До этого мы подготовили колонки **Жанры** и **Год выпуска фильма**. Давайте немного изменим формулу, чтобы учитывать жанр фильма и год выпуска. Формула должна принимать дополнительный параметр **Жанр**, если фильм содержит все жанры, которые выбирает пользователь. Только тогда фильм попадает в финальный рекомендованный рейтинг.

In [74]:
def recommend_by_genre(input_films, n_items, genres):
    rank = input_films.copy()
    
    C = rank.vote_average.mean()
    Q = rank.vote_count.quantile(0.95)
    print("Minimum number of votes: {}. Average overall rating: {}".format(int(Q),C))
    rank['formula'] = rank.apply(lambda x: the_best_formula(x, Q, C), axis=1)
    rank.sort_values('formula', ascending=False, inplace=True)
    
    rank['contains_genre'] = [all( y in x for y in genres) for x in rank.genres]
    rank = rank[rank.contains_genre == True]
    rank.drop('contains_genre', axis=1, inplace=True)

    return rank.head(n_items).reset_index(drop=True)

In [76]:
recommend_by_genre(dataset, 10, genres=['Comedy', 'Horror'])

Minimum number of votes: 433. Average overall rating: 5.618217011635541


,title,year,vote_count,vote_average,genres,formula
0,The Empire Strikes Back,1980.0,5998,8.2,"[Adventure, Action, Science Fiction]",8.025831
1,Inception,2010.0,14075,8.1,"[Action, Thriller, Science Fiction, Mystery, A...",8.025780
2,Interstellar,2014.0,11187,8.1,"[Adventure, Drama, Science Fiction]",8.007335
3,Star Wars,1977.0,6778,8.1,"[Adventure, Action, Science Fiction]",7.950685
4,Back to the Future,1985.0,6239,8.0,"[Adventure, Comedy, Science Fiction, Family]",7.845126
5,Guardians of the Galaxy,2014.0,10014,7.9,"[Action, Science Fiction, Adventure]",7.805238
6,Return of the Jedi,1983.0,4763,7.9,"[Adventure, Action, Science Fiction]",7.709489
7,2001: A Space Odyssey,1968.0,3075,7.9,"[Science Fiction, Mystery, Adventure]",7.617842
8,The Martian,2015.0,7442,7.6,"[Drama, Adventure, Science Fiction]",7.490819
9,Captain America: The Winter Soldier,2014.0,5881,7.6,"[Action, Adventure, Science Fiction]",7.463831


Супер, теперь мы можем выбирать произвольные жанры фильмов.

То же самое сделаем с параметром **год выпуска фильма**. У формулы появляется ещё один параметр, если фильм был выпущен в указанный год или позже, тогда он попадает в финальную выдачу.

In [77]:
def recommend_by_genre_and_year(input_films, n_items, min_votes, genres, not_older_than):
    
    rank = input_films.copy()
    
    C = rank.vote_average.mean()
    Q = rank.vote_count.quantile(0.95)
    print("Minimum number of votes: {}. Average overall rating: {}".format(int(Q),C))
    rank['formula'] = rank.apply(lambda x: the_best_formula(x, Q, C), axis=1)
    rank.sort_values('formula', ascending=False, inplace=True)
    
    rank['contains_genre'] = [all( y in x for y in genres) for x in rank.genres]
    rank = rank[(rank.contains_genre == True) & (rank.year >= not_older_than)]
    rank.drop('contains_genre', axis=1, inplace=True)
    
    return rank.head(n_items).reset_index(drop=True)

In [79]:
recommend_by_genre_and_year(dataset, 15, 150, genres=['Comedy', 'Horror'], not_older_than=2011)

Minimum number of votes: 433. Average overall rating: 5.618217011635541


,title,year,vote_count,vote_average,genres,formula
0,What We Do in the Shadows,2014.0,779,7.4,"[Comedy, Horror]",6.762589
1,Warm Bodies,2013.0,2698,6.4,"[Horror, Comedy, Romance]",6.291690
2,The Final Girls,2015.0,371,6.6,"[Horror, Comedy]",6.070747
3,Goosebumps,2015.0,1022,6.2,"[Adventure, Horror, Comedy]",6.026612
4,Housebound,2014.0,362,6.5,"[Horror, Comedy, Thriller]",6.019279
5,Dead Snow 2: Red vs. Dead,2014.0,199,6.7,"[Horror, Action, Comedy]",5.958357
6,The Voices,2014.0,578,6.2,"[Comedy, Crime, Horror, Thriller]",5.950533
7,Scouts Guide to the Zombie Apocalypse,2015.0,526,6.2,"[Comedy, Horror]",5.937019
8,John Dies at the End,2012.0,257,6.3,"[Horror, Comedy]",5.871826
9,Fright Night,2011.0,617,6.0,"[Horror, Comedy]",5.842368


По аналогии ты можешь добавлять любые другие параметры в свою рекомендательную систему.
____

In [106]:
def recommend_adult_by_year(input_films, n_items, min_votes, not_older_than, adult):
    
    rank = input_films.copy()
    
    C = rank.vote_average.mean()
    Q = rank.vote_count.quantile(0.95)
    print("Minimum number of votes: {}. Average overall rating: {}".format(int(Q),C))
    rank['formula'] = rank.apply(lambda x: the_best_formula(x, Q, C), axis=1)
    rank.sort_values('formula', ascending=False, inplace=True)
    
    rank = rank[(rank.year >= not_older_than) & (rank.adult == adult)]
    
    return rank.head(n_items).reset_index(drop=True)

In [107]:
recommend_adult_by_year(dataset, 15, 150, not_older_than=2011, adult = True)

Minimum number of votes: 433. Average overall rating: 5.618217011635541


,title,year,vote_count,vote_average,genres,adult,formula
0,Adulterers,2016.0,16,5.2,"[Thriller, Crime, Drama]",True,5.603344
1,Diet of Sex,2014.0,12,4.0,"[Comedy, Drama, Romance]",True,5.574668


# Итоги

Ты разработал простую рекомендательную систему. Она строится на предположении, что фильмы, которые нравятся большинству, понравятся и тебе. Конечно, это не совсем так, поэтому в следующих заданиях мы учтём персональные вкусы и запрограммируем более интеллектуальные рекомендательные системы.